# Simple implementation af Kalman filteret
Koden er nu skrevet sådan at $F$ er afhængig af tid, da vores koordinater indgår i dens udregning. Vi skal have nogle anre ligninger i spil for at kunne lave den uden det
<br>
<b>OBS OBS OBS OBS OBS</b> <br>
<b>Læs ovenfor</b>
\begin{align*}
    \intertext{kinematic and observations model}
    \mathbf{x}[n]&=\Phi \mathbf{x}[n-1] + \mathbf{u}[n], \\
    \mathbf{z}[n]&=\mathbf{x}[n] + \mathbf{w}[n],\\
    \intertext{the prediction and prediction mean square error,}
    \hat{\mathbf{x}}[n|n-1]&=\Phi\hat{\mathbf{x}}[n-1|n-1], \\
    M[n|n-1]&=\Phi M[n-1|n-1]\Phi^\top + S_{\mathbf{u}}, \\
    \intertext{the Kalman gain,}
    K[n]&=M[n|n-1](S_{\mathbf{w}}+M[n|n-1])^{-1},\\
    \intertext{and the correction and estimate mean square error,}
    \hat{\mathbf{x}}[n|n]&=\hat{\mathbf{x}}[n|n-1]+K[n](\mathbf{z}[n]-\hat{\mathbf{x}}[n|n-1]),\\
    M[n|n]&=(I-K[n])M[n|n-1].
\end{align*}
<br>

In [45]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import importlib
from scipy.signal import resample
sys.path.insert(1, os.getcwd())
import tracking as tr
importlib.reload(tr)

<module 'tracking' from '/home/anders/Documents/AAU/P6/tracking/tracking.py'>

## Hent dataen for sig
Den her blok kode henter bare dataen ned, hvis man skulle have lyst til det. Lige nu er der ikke en praktisk årsag til det, udover at gemme de filnavne vi skal bruge

In [39]:
# True orbits
true50_names = ["snr50/entireOrbit" + str(i) + ".txt" for i in range(1, 6)]
true30_names = ["snr30/entireOrbit" + str(i) + ".txt" for i in range(1, 6)]
true10_names = ["snr10/entireOrbit" + str(i) + ".txt" for i in range(1, 6)]
true50 = [tr.import_data(name) for name in true50_names]
true30 = [tr.import_data(name) for name in true30_names]
true10 = [tr.import_data(name) for name in true10_names]

# True detections
detect50_names = ["snr50/truth" + str(i) + ".txt" for i in range(1, 6)]
detect30_names = ["snr30/truth" + str(i) + ".txt" for i in range(1, 6)]
detect10_names = ["snr10/truth" + str(i) + ".txt" for i in range(1, 6)]
detect50 = [tr.import_data(name) for name in detect50_names]
detect30 = [tr.import_data(name) for name in detect30_names]
detect10 = [tr.import_data(name) for name in detect10_names]

# False detections
false50 = tr.import_data("snr50/false.txt")
false30 = tr.import_data("snr30/false.txt")
false10 = tr.import_data("snr10/false.txt")

## Estimater og definitioner
Her laver vi estimater af vores noise covariance matricer og startgæt for vores state. Vi importerer også dataen, og udregner dt-vektorene for vores tracks som gives til Kalman filteret

In [48]:
true50_state = [tr.velocity_algo(name, true_orbit=True) for name in true50_names]
true30_state = [tr.velocity_algo(name, true_orbit=True) for name in true30_names]
true10_state = [tr.velocity_algo(name, true_orbit=True) for name in true10_names]